In [172]:
import respy as rp
import pandas as pd
import numpy as np

from auxiliary import *

pd.set_option('display.max_rows', 1000)

In [173]:
params, options, data = rp.get_example_model("kw_94_two")

num_occupations, n_periods = 3, 3

In [184]:
if True:
    n_periods = 3

    for model in ["kw_97_basic"]:

        params, options, data = rp.get_example_model(model)

        for num_occupations in range(1, 3):
            params_occ, options_occ = update_model_specification(params, options, num_occupations)

            choices = _get_choices_occupations(params_occ)[0]

            options["n_periods"] = n_periods

            simulate = rp.get_simulate_func(params_occ, options)
            simulate(params_occ)

            np.testing.assert_equal(num_occupations + 2, len(choices))

['aa', 'blue_collar', 'military', 'white_collar', 'school', 'home']


/home/peisenha/local/anaconda3/envs/respy-development/lib/python3.7/site-packages/respy/pre_processing/model_processing.py:590: UserWarning: The probabilities for parameter group \binitial_exp_school_([0-9]+)\b do not sum to one.
  category=UserWarning,
/home/peisenha/local/anaconda3/envs/respy-development/lib/python3.7/site-packages/respy/pre_processing/model_processing.py:590: UserWarning: The probabilities for parameter group \binitial_exp_school_([0-9]+)\b do not sum to one.
  category=UserWarning,
/home/peisenha/local/anaconda3/envs/respy-development/lib/python3.7/site-packages/respy/pre_processing/model_processing.py:590: UserWarning: The probabilities for parameter group \binitial_exp_school_([0-9]+)\b do not sum to one.
  category=UserWarning,
/home/peisenha/local/anaconda3/envs/respy-development/lib/python3.7/site-packages/respy/pre_processing/model_processing.py:590: UserWarning: The probabilities for parameter group \binitial_exp_school_([0-9]+)\b do not sum to one.
  catego

AssertionError: 
Items are not equal:
 ACTUAL: 3
 DESIRED: 6